# Предобработка данных для рекомендательной системы

In [1]:
import pandas as pd


class Constants:
    USER_ID = "user_id"
    ITEM_ID = "item_id"
    TIMESTAMP = "time"

    TRANSACTIONS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/transactions_train.csv"
    CUSTOMERS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/customers.csv"
    ARTICLES_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/articles.csv"

    RESULT_TRANSACTIONS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/processed_transactions_train.csv"
    RESULT_CUSTOMERS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/processed_customers_train.csv"
    RESULT_ARTICLES_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/processed_articles_train.csv"


In [2]:
def prepare_filtered_dataset(
    transactions: pd.DataFrame,
    num_customers: int = 100,
    num_articles: int = 100,
    min_articles_per_user: int = 5,
    min_users_per_article: int = 5,
) -> tuple:
    """
    Отбираем сбалансированный набор данных:
    - num_customers: количество пользователей для отбора
    - min_articles_per_user: минимальное количество покупок у пользователя
    - min_users_per_article: минимальное количество покупателей у товара
    """

    # Фильтрация товаров
    article_popularity = transactions[Constants.ITEM_ID].value_counts()
    popular_articles = article_popularity[article_popularity >= min_users_per_article]

    # Отбираем топ-N самых активных пользователей
    selected_items = popular_articles.head(num_articles).index

    # Окончательный набор данных
    filtered_data = transactions[
        transactions[Constants.ITEM_ID].isin(selected_items)
    ]

    # Первичная фильтрация пользователей
    user_purchase_counts = filtered_data[Constants.USER_ID].value_counts()
    active_users = user_purchase_counts[user_purchase_counts >= min_articles_per_user]

    # Отбираем топ-N самых активных пользователей
    selected_users = active_users.head(num_customers).index

    # Фильтруем транзакции только по выбранным пользователям
    final_data = filtered_data[filtered_data[Constants.USER_ID].isin(selected_users)]
    final_data = final_data.reset_index(drop=True)

    print(f"Исходный размер данных: {len(transactions)} транзакций")
    print(f"Отобрано пользователей: {len(selected_users)}")
    print(f"Отобрано товаров: {len(selected_items)}")
    print(f"Финальный размер данных: {len(final_data)} транзакций")

    return final_data, selected_users, selected_items


In [3]:
transactions = pd.read_csv(Constants.TRANSACTIONS_PATH)
transactions = transactions.rename(
    columns={
        "t_dat": Constants.TIMESTAMP,
        "customer_id": Constants.USER_ID,
        "article_id": Constants.ITEM_ID,
    }
)
transactions[Constants.USER_ID] = transactions[Constants.USER_ID].astype(str)
transactions[Constants.ITEM_ID] = transactions[Constants.ITEM_ID].astype(str)

customers = pd.read_csv(Constants.CUSTOMERS_PATH)
customers = (
    customers.rename(
        columns={
            "customer_id": Constants.USER_ID,
            "Active": "is_active",
            "age": "age",
        }
    )
    .drop(columns=["FN", "postal_code"])
)
customers[Constants.USER_ID] = customers[Constants.USER_ID].astype(str)

articles = pd.read_csv(Constants.ARTICLES_PATH)
articles = (
    articles.rename(
        columns={"article_id": Constants.ITEM_ID}
    )
    [[
        Constants.ITEM_ID,
        "prod_name",
        "product_type_name",
        "product_group_name",
        "colour_group_name",
        "detail_desc",
    ]]
)
articles[Constants.ITEM_ID] = articles[Constants.ITEM_ID].astype(str)

In [4]:
articles.head()

,item_id,prod_name,product_type_name,product_group_name,colour_group_name,detail_desc
0,108775015,Strap top,Vest top,Garment Upper body,Black,Jersey top with narrow shoulder straps.
1,108775044,Strap top,Vest top,Garment Upper body,White,Jersey top with narrow shoulder straps.
2,108775051,Strap top (1),Vest top,Garment Upper body,Off White,Jersey top with narrow shoulder straps.
3,110065001,OP T-shirt (Idro),Bra,Underwear,Black,"Microfibre T-shirt bra with underwired, moulde..."
4,110065002,OP T-shirt (Idro),Bra,Underwear,White,"Microfibre T-shirt bra with underwired, moulde..."


In [5]:
customers.head()

,user_id,is_active,club_member_status,fashion_news_frequency,age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,ACTIVE,NONE,49.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,ACTIVE,NONE,25.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,ACTIVE,NONE,24.0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,ACTIVE,NONE,54.0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,ACTIVE,Regularly,52.0


In [6]:
transactions.head()

,time,user_id,item_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [7]:
filtered_transactions, selected_users, selected_items = prepare_filtered_dataset(transactions=transactions)
filtered_customers = customers[customers[Constants.USER_ID].isin(selected_users)]
filtered_articles = articles[articles[Constants.ITEM_ID].isin(selected_items)]

Исходный размер данных: 31788324 транзакций
Отобрано пользователей: 100
Отобрано товаров: 100
Финальный размер данных: 5542 транзакций


In [8]:
filtered_transactions.to_parquet(Constants.RESULT_TRANSACTIONS_PATH, index=False)
filtered_customers.to_parquet(Constants.RESULT_CUSTOMERS_PATH, index=False)
filtered_articles.to_parquet(Constants.RESULT_ARTICLES_PATH, index=False)

# Загрузка предобработанных данных в PostgreSQL

In [9]:
import pandas as pd
import psycopg2
from io import StringIO

# Подключение к PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="postgres"
)
cur = conn.cursor()

def load_table_from_dataframe(df, table_name):
    # Создаем буфер для копирования данных
    buffer = StringIO()
    df.to_csv(buffer, index=False, header=False, sep='\t')
    buffer.seek(0)
    
    try:
        # Для таблицы transactions используем обычный TRUNCATE
        if table_name == 'recsys.transactions':
            cur.execute(f"TRUNCATE TABLE {table_name}")
        # Для таблиц с внешними ключами используем TRUNCATE CASCADE
        else:
            cur.execute(f"TRUNCATE TABLE {table_name} CASCADE")
        
        # Копируем данные из буфера в таблицу
        cur.copy_from(buffer, table_name, sep='\t', null='')
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"Ошибка при загрузке в таблицу {table_name}: {e}")

# Создаем таблицы (если их нет)
def create_tables():
    # Сначала создаем схему, если она не существует
    try:
        cur.execute("CREATE SCHEMA IF NOT EXISTS recsys;")
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"Ошибка при создании схемы: {e}")

    # Сначала удаляем существующие таблицы (если нужно)
    commands_drop = (
        "DROP TABLE IF EXISTS recsys.transactions CASCADE",
        "DROP TABLE IF EXISTS recsys.customers CASCADE",
        "DROP TABLE IF EXISTS recsys.articles CASCADE"
    )
    
    # Затем создаем новые таблицы
    commands_create = (
        """
        CREATE TABLE IF NOT EXISTS recsys.articles (
            item_id TEXT PRIMARY KEY,
            prod_name TEXT,
            product_type_name TEXT,
            product_group_name TEXT,
            colour_group_name TEXT,
            detail_desc TEXT
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS recsys.customers (
            user_id TEXT PRIMARY KEY,
            is_active FLOAT,
            club_member_status TEXT,
            fashion_news_frequency TEXT,
            age FLOAT
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS recsys.transactions (
            time TIMESTAMP,
            user_id TEXT,
            item_id TEXT,
            price FLOAT,
            sales_channel_id INTEGER,
            FOREIGN KEY (user_id) REFERENCES recsys.customers (user_id),
            FOREIGN KEY (item_id) REFERENCES recsys.articles (item_id)
        )
        """
    )
    
    try:
        # Удаляем таблицы (если существуют)
        for command in commands_drop:
            cur.execute(command)
        
        # Создаем новые таблицы
        for command in commands_create:
            cur.execute(command)
        
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"Ошибка при создании таблиц: {e}")


if __name__ == "__main__":
    # Создаем таблицы
    create_tables()
    
    # Важно загружать данные в правильном порядке: сначала справочники, потом транзакции
    load_table_from_dataframe(articles, 'recsys.articles')
    load_table_from_dataframe(customers, 'recsys.customers')
    load_table_from_dataframe(transactions, 'recsys.transactions')
    
    cur.close()
    conn.close()
    print("Данные успешно загружены в PostgreSQL в схему recsys")


OperationalError: connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"
